# Ejercicio NoSQL con Dynamodb-local y MongoDB

In [4]:
from faker import Faker # crar datos ficticios python -m pip install faker

import boto3  # drive Dynamodb python -m pip install boto3
from botocore.exceptions import ClientError

from pymongo import MongoClient # cliente Mongodb python -m pip install pymongo

import random
import time

## Crear Datos Ficticios

In [5]:
fake = Faker()
faker_seed = 123
Faker.seed(faker_seed)

In [9]:
# ---------- Generar datos ficticios ----------
def generar_productos(n=10):
    productos = []
    for i in range(1, n + 1):
        productos.append({
            "Id": i,
            "Nombre": fake.word().capitalize(),
            "Precio": round(random.uniform(1.0, 100.0), 2),
            "Stock": random.randint(10, 200)
        })
    return productos

In [10]:
productos = generar_productos(50)

In [11]:
productos

[{'Id': 1, 'Nombre': 'American', 'Precio': 80.93, 'Stock': 11},
 {'Id': 2, 'Nombre': 'Hundred', 'Precio': 16.3, 'Stock': 33},
 {'Id': 3, 'Nombre': 'Not', 'Precio': 62.9, 'Stock': 117},
 {'Id': 4, 'Nombre': 'Or', 'Precio': 32.61, 'Stock': 16},
 {'Id': 5, 'Nombre': 'Go', 'Precio': 71.91, 'Stock': 151},
 {'Id': 6, 'Nombre': 'Grow', 'Precio': 49.91, 'Stock': 87},
 {'Id': 7, 'Nombre': 'Together', 'Precio': 91.85, 'Stock': 84},
 {'Id': 8, 'Nombre': 'Area', 'Precio': 63.84, 'Stock': 159},
 {'Id': 9, 'Nombre': 'Collection', 'Precio': 23.05, 'Stock': 39},
 {'Id': 10, 'Nombre': 'Central', 'Precio': 9.34, 'Stock': 194},
 {'Id': 11, 'Nombre': 'Green', 'Precio': 77.3, 'Stock': 54},
 {'Id': 12, 'Nombre': 'Option', 'Precio': 99.85, 'Stock': 48},
 {'Id': 13, 'Nombre': 'Goal', 'Precio': 40.86, 'Stock': 168},
 {'Id': 14, 'Nombre': 'Science', 'Precio': 25.85, 'Stock': 30},
 {'Id': 15, 'Nombre': 'Else', 'Precio': 74.66, 'Stock': 90},
 {'Id': 16, 'Nombre': 'Common', 'Precio': 30.92, 'Stock': 68},
 {'Id': 1

In [12]:
import json

with open('productos_2.json', 'w') as f:
    json.dump(productos, f, indent=4)



## Guardar los Datos en MongoDB

In [13]:
# ---------- MongoDB ----------
cliente_mongo = MongoClient("mongodb://localhost:27017/")
db_mongo = cliente_mongo["tienda_2"]
coleccion = db_mongo["productos"]
coleccion.drop()  # Limpiar colección antes de insertar

In [15]:
# ---------- Guardar en MongoDB ----------
def guardar_en_mongodb(productos):
    coleccion.insert_many(productos)
    print(f"{len(productos)} productos insertados en MongoDB.")

In [16]:
guardar_en_mongodb(productos)

50 productos insertados en MongoDB.


## Guardar los Datos en una Tabla de Dynamodb

In [17]:
cliente = boto3.client(
        "dynamodb",
        endpoint_url="http://localhost:8000",  # 🔸 DynamoDB Local
        region_name="us-west-2",               # 🔹 Requerido por boto3 (puede ser cualquiera)
        aws_access_key_id="fakeMyKeyId",       # 🔹 Cualquier valor
        aws_secret_access_key="fakeSecretKey"  # 🔹 Cualquier valor
    )

In [18]:
table_name = "Productos_2"

In [19]:
def create_table_db(table_name: str, **kwargs):
    """
    Crea una tabla en DynamoDB si no existe.

    Parámetros:
    -----------
    table_name : str
        El nombre de la tabla a crear.

    **kwargs :
        Argumentos adicionales requeridos por boto3 para crear la tabla.
        Ejemplo típico:
            - KeySchema
            - AttributeDefinitions
            - ProvisionedThroughput

    Retorna:
    --------
    dict
        La respuesta del cliente si la tabla fue creada, o un mensaje si ya existía.

    Nota:
    -----
    Esta función asume que `cliente` ya está definido y configurado para acceder a DynamoDB
    (local o en la nube).
    """

    try:
        # Verificamos si la tabla ya existe
        cliente.describe_table(TableName=table_name)
        return {'Message': f'La tabla "{table_name}" ya existe. No se creó de nuevo.'}
    except cliente.exceptions.ResourceNotFoundException:
        # Si no existe, procedemos a crearla
        response = cliente.create_table(TableName=table_name, **kwargs)

        # Esperamos hasta que esté creada completamente
        cliente.get_waiter("table_exists").wait(TableName=table_name)

        return response


In [20]:
TABLE_SCHEMA = {
    "KeySchema": [
        {
            "AttributeName": "Id",
            "KeyType": "HASH"  # Clave primaria
        }
    ],
    "AttributeDefinitions": [
        {
            "AttributeName": "Id",
            "AttributeType": "N"  # Tipo numérico
        },
    ],
    "ProvisionedThroughput": {
        "ReadCapacityUnits": 5,
        "WriteCapacityUnits": 5
    }
}

In [21]:
respuesta = create_table_db(table_name, **TABLE_SCHEMA)

In [22]:
respuesta.keys()

dict_keys(['TableDescription', 'ResponseMetadata'])

In [23]:
respuesta

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'Id',
    'AttributeType': 'N'}],
  'TableName': 'Productos_2',
  'KeySchema': [{'AttributeName': 'Id', 'KeyType': 'HASH'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2025, 6, 10, 20, 10, 53, 19000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 21, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 21, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/Productos_2',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': '2f8d8059-983f-4820-a860-d60fa7eb2cd4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Jetty(12.0.14)',
   'date': 'Tue, 10 Jun 2025 23:10:52 GMT',
   'x-amzn-requestid': '2f8d8059-983f-4820-a860-d60fa7eb2cd4',
  

In [24]:
cliente.list_tables()['TableNames']

['Data-Engineering-ProductCatalog', 'Productos', 'Productos_2']

In [25]:
# ---------- Guardar en DynamoDB ----------
def guardar_en_dynamodb(cliente,tabla, productos):
    for item in productos:
        print(item)
        cliente.put_item(
        TableName = tabla, 
        Item      = item)
    print(f"{len(productos)} productos insertados en la {tabla} en DynamoDB.")

In [26]:
guardar_en_dynamodb(cliente,"Productos", productos)

{'Id': 1, 'Nombre': 'American', 'Precio': 80.93, 'Stock': 11, '_id': ObjectId('6848ba0d471d6dbebeeb1b22')}


ParamValidationError: Parameter validation failed:
Invalid type for parameter Item.Id, value: 1, type: <class 'int'>, valid types: <class 'dict'>
Invalid type for parameter Item.Nombre, value: American, type: <class 'str'>, valid types: <class 'dict'>
Invalid type for parameter Item.Precio, value: 80.93, type: <class 'float'>, valid types: <class 'dict'>
Invalid type for parameter Item.Stock, value: 11, type: <class 'int'>, valid types: <class 'dict'>
Invalid type for parameter Item._id, value: 6848ba0d471d6dbebeeb1b22, type: <class 'bson.objectid.ObjectId'>, valid types: <class 'dict'>

In [27]:
productos[0]

{'Id': 1,
 'Nombre': 'American',
 'Precio': 80.93,
 'Stock': 11,
 '_id': ObjectId('6848ba0d471d6dbebeeb1b22')}

Lo que te pasó es un efecto secundario común al trabajar con PyMongo: MongoDB modifica in-place (en el lugar) tus objetos de Python agregándoles el campo "_id" automáticamente cuando usás insert_many() o insert_one().

- 🔍 ¿Qué pasó exactamente?
Supongamos que tenías esta lista:

```python
productos = [
    {"Id": 1, "Nombre": "Regla", "Precio": 2.99, "Stock": 75},
    {"Id": 2, "Nombre": "Lapicera", "Precio": 5.50, "Stock": 100},
]
```

Cuando hacés:

```python
coleccion.insert_many(productos)
```

MongoDB agrega una nueva clave "_id" a cada uno de esos diccionarios en la lista. Ahora tu lista quedó así:

```python
[
    {"Id": 1, "Nombre": "Regla", "Precio": 2.99, "Stock": 75, "_id": ObjectId(...)},
    {"Id": 2, "Nombre": "Lapicera", "Precio": 5.50, "Stock": 100, "_id": ObjectId(...)},
]
```

💥 ¿Y por qué eso rompe DynamoDB?

DynamoDB no permite claves que no estén esperadas, o tipos como ObjectId, que MongoDB agrega por defecto y no son válidos en DynamoDB.

✅ Soluciones

Opción 1: Insertar una copia de la lista en MongoDB
```python
import copy

# Copiamos la lista para que Mongo no modifique la original
productos_para_mongo = copy.deepcopy(productos)
coleccion.insert_many(productos_para_mongo)
```

Opción 2: Borrar la clave _id antes de insertar en DynamoDB

```python
for item in productos:
    item.pop("_id", None)  # Elimina la clave _id si existe
    client.put_item(
        TableName="Productos",
        Item={
            "Id": {"N": str(item["Id"])},
            "Nombre": {"S": item["Nombre"]},
            "Precio": {"N": str(item["Precio"])},
            "Stock": {"N": str(item["Stock"])}
        }
    )
````

🧠 Recomendación

Siempre que uses datos con MongoDB y otra base de datos al mismo tiempo, asegurate de:

Trabajar sobre copias (deepcopy) cuando los insertes en Mongo.

O limpiar los "_id" antes de usarlos en otra parte.

In [29]:
datos = json.load(open('productos_2.json', 'r'))
datos[0]

{'Id': 1, 'Nombre': 'American', 'Precio': 80.93, 'Stock': 11}

In [30]:
res = guardar_en_dynamodb(cliente,"Productos",  datos)

{'Id': 1, 'Nombre': 'American', 'Precio': 80.93, 'Stock': 11}


ParamValidationError: Parameter validation failed:
Invalid type for parameter Item.Id, value: 1, type: <class 'int'>, valid types: <class 'dict'>
Invalid type for parameter Item.Nombre, value: American, type: <class 'str'>, valid types: <class 'dict'>
Invalid type for parameter Item.Precio, value: 80.93, type: <class 'float'>, valid types: <class 'dict'>
Invalid type for parameter Item.Stock, value: 11, type: <class 'int'>, valid types: <class 'dict'>

> En DynamoDB, cuando usás el cliente de boto3 (es decir, client = boto3.client('dynamodb', ...)), los datos que pasás a put_item deben estar tipados explícitamente, es decir, cada valor debe estar envuelto en un diccionario que indique su tipo:

🔥 Error actual
Estás pasando algo como esto:

```python
{
    'Id': 1,
    'Nombre': 'Area',
    'Precio': 56.38,
    'Stock': 144
}
```

Pero DynamoDB espera esto:

```python
{
    'Id': {'N': '1'},
    'Nombre': {'S': 'Area'},
    'Precio': {'N': '56.38'},
    'Stock': {'N': '144'}
}
```

✅ Solución: convertir tus ítems al formato esperado
Podés hacer una función que convierta tus diccionarios Python normales al formato que necesita DynamoDB:

```python
def convertir_a_item_dynamodb(producto: dict) -> dict:
    item_dynamodb = {}
    for clave, valor in producto.items():
        if isinstance(valor, str):
            item_dynamodb[clave] = {'S': valor}
        elif isinstance(valor, (int, float)):
            item_dynamodb[clave] = {'N': str(valor)}  # Siempre como string
        else:
            raise ValueError(f"Tipo de dato no soportado para la clave '{clave}': {type(valor)}")
    return item_dynamodb
```
Y luego usás eso en tu inserción:

 ```python
for producto in productos:
    item = convertir_a_item_dynamodb(producto)
    cliente.put_item(
        TableName="Productos",
        Item=item
    )
```

🧠 Importante

DynamoDB requiere que los tipos estén indicados ('S' para string, 'N' para número, 'BOOL', 'L', 'M', etc.).

En el modo client, vos te encargás de esa estructura.

Si querés evitar este detalle, podrías usar resource, que lo hace más transparente, pero es opcional.

In [38]:
conn = boto3.resource(
    'dynamodb',
    region_name='us-west-2',
    endpoint_url='http://localhost:8000', # importante para conexión local
    aws_access_key_id='fakeMyKeyId',
    aws_secret_access_key='fakeSecretAccessKey')

In [39]:
list(conn.tables.all())

[dynamodb.Table(name='Data-Engineering-ProductCatalog'),
 dynamodb.Table(name='Productos'),
 dynamodb.Table(name='Productos_2')]

In [63]:
tabla = conn.Table('Productos_2')

In [64]:
type(tabla)

boto3.resources.factory.dynamodb.Table

In [65]:
tabla

dynamodb.Table(name='Productos_2')

In [43]:
def guardar_en_dynamodb_con_resorce(tabla, productos):
    for item in productos:
        print(item)
        tabla.put_item(Item=item)
    print(f"{len(productos)} productos insertados en la {tabla} en DynamoDB.")

In [44]:
guardar_en_dynamodb_con_resorce(tabla, datos)

{'Id': 1, 'Nombre': 'American', 'Precio': 80.93, 'Stock': 11}


TypeError: Float types are not supported. Use Decimal types instead.

🛠️ ¿Por qué pasa esto?

> DynamoDB (y boto3) requieren que los números con decimales sean instancias de Decimal, no de float, porque:

- float en Python puede tener errores de precisión binaria.
- Decimal garantiza precisión exacta en los valores almacenados.

✅ Solución: convertir los float a Decimal

```python
from decimal import Decimal

producto = {
    'Id': 1,
    'Nombre': 'Area',
    'Precio': Decimal('56.38'),  # 👈 así
    'Stock': 144
}
```

🧹 Tip para convertir todos los float en un diccionario:

```python
from decimal import Decimal

def convertir_floats_a_decimal(diccionario):
    return {
        k: Decimal(str(v)) if isinstance(v, float) else v
        for k, v in diccionario.items()
    }

producto = {'Id': 1, 'Nombre': 'Area', 'Precio': 56.38, 'Stock': 144}
producto = convertir_floats_a_decimal(producto)
tabla.put_item(Item=producto)
```

✅ BONUS: Si querés convertir listas de productos

```python
productos = [
    {'Id': 1, 'Nombre': 'Area', 'Precio': 56.38, 'Stock': 144},
    {'Id': 2, 'Nombre': 'H2Oh!', 'Precio': 45.90, 'Stock': 67}
]

def convertir_lista(productos):
    return [convertir_floats_a_decimal(p) for p in productos]

productos_convertidos = convertir_lista(productos)

for producto in productos_convertidos:
    tabla.put_item(Item=producto)
```

In [66]:
from decimal import Decimal

def convertir_floats_a_decimal(diccionario):
    return {
        k: Decimal(str(v)) if isinstance(v, float) else v
        for k, v in diccionario.items()
    }


In [67]:
DATOS_DECIMAL = [convertir_floats_a_decimal(producto) for producto in datos]

In [68]:
DATOS_DECIMAL[0]

{'Id': 1, 'Nombre': 'American', 'Precio': Decimal('80.93'), 'Stock': 11}

In [69]:
guardar_en_dynamodb_con_resorce(tabla, DATOS_DECIMAL)

{'Id': 1, 'Nombre': 'American', 'Precio': Decimal('80.93'), 'Stock': 11}
{'Id': 2, 'Nombre': 'Hundred', 'Precio': Decimal('16.3'), 'Stock': 33}
{'Id': 3, 'Nombre': 'Not', 'Precio': Decimal('62.9'), 'Stock': 117}
{'Id': 4, 'Nombre': 'Or', 'Precio': Decimal('32.61'), 'Stock': 16}
{'Id': 5, 'Nombre': 'Go', 'Precio': Decimal('71.91'), 'Stock': 151}
{'Id': 6, 'Nombre': 'Grow', 'Precio': Decimal('49.91'), 'Stock': 87}
{'Id': 7, 'Nombre': 'Together', 'Precio': Decimal('91.85'), 'Stock': 84}
{'Id': 8, 'Nombre': 'Area', 'Precio': Decimal('63.84'), 'Stock': 159}
{'Id': 9, 'Nombre': 'Collection', 'Precio': Decimal('23.05'), 'Stock': 39}
{'Id': 10, 'Nombre': 'Central', 'Precio': Decimal('9.34'), 'Stock': 194}
{'Id': 11, 'Nombre': 'Green', 'Precio': Decimal('77.3'), 'Stock': 54}
{'Id': 12, 'Nombre': 'Option', 'Precio': Decimal('99.85'), 'Stock': 48}
{'Id': 13, 'Nombre': 'Goal', 'Precio': Decimal('40.86'), 'Stock': 168}
{'Id': 14, 'Nombre': 'Science', 'Precio': Decimal('25.85'), 'Stock': 30}
{'Id': 

## Querys 

### 🔍 1. Consulta por Id


#### MongoDB

In [49]:
from pymongo import MongoClient 

In [55]:
cliente_mongo = MongoClient("mongodb://localhost:27017/")
db_mongo = cliente_mongo["tienda_2"]
coleccion = db_mongo["productos"]


In [56]:
def buscar_producto_mongo(db, id_producto):
    return db.productos.find_one({"Id": id_producto})


In [57]:
mongo_1 = buscar_producto_mongo(db_mongo, 1)

In [58]:
type(mongo_1)

dict

In [59]:
import json
print(json.dumps(mongo_1, indent=4, default=str))  # Convertir a JSON para mejor visualización

{
    "_id": "6848ba0d471d6dbebeeb1b22",
    "Id": 1,
    "Nombre": "American",
    "Precio": 80.93,
    "Stock": 11
}


#### DynamoDB

In [70]:
import boto3  # drive Dynamodb python -m pip install boto3
from botocore.exceptions import ClientError

In [71]:
db_dynamo = boto3.resource(
    'dynamodb',
    region_name='us-west-2',
    endpoint_url='http://localhost:8000', # importante para conexión local
    aws_access_key_id='fakeMyKeyId',
    aws_secret_access_key='fakeSecretAccessKey')

In [72]:
tabla = db_dynamo.Table('Productos_2')
tabla

dynamodb.Table(name='Productos_2')

In [73]:
def buscar_producto_dynamo(tabla, id_producto):
    response = tabla.get_item(Key={"Id": id_producto})
    return response.get("Item")

In [74]:
dynamo_1 = buscar_producto_dynamo(tabla, 1)

In [75]:
type(dynamo_1)

dict

In [76]:
print(json.dumps(dynamo_1, indent=4, default=str))

{
    "Nombre": "American",
    "Id": "1",
    "Stock": "11",
    "Precio": "80.93"
}


### 🔍 2. Consulta por nombre (parcial o exacto)



#### MongoDB (regex para búsqueda parcial)


In [77]:
def buscar_por_nombre_mongo(db, nombre):
    return list(db.productos.find({"Nombre": {"$regex": nombre, "$options": "i"}}))


In [84]:
res = buscar_por_nombre_mongo(db_mongo, "Ame")


In [85]:
res

[{'_id': ObjectId('6848ba0d471d6dbebeeb1b22'),
  'Id': 1,
  'Nombre': 'American',
  'Precio': 80.93,
  'Stock': 11}]

In [87]:
buscar_por_nombre_mongo(db_mongo, "American")

[{'_id': ObjectId('6848ba0d471d6dbebeeb1b22'),
  'Id': 1,
  'Nombre': 'American',
  'Precio': 80.93,
  'Stock': 11}]

#### DynamoDB (scan con filtro de igualdad)


DynamoDB no permite búsquedas por atributos que no son clave sin hacer un scan.


In [89]:
from boto3.dynamodb.conditions import Attr

def buscar_por_nombre_dynamo(tabla, nombre):
    response = tabla.scan(
        FilterExpression=Attr("Nombre").eq(nombre)
    )
    return response.get("Items", [])


⚠️ Para búsquedas parciales o por regex como en Mongo, deberías usar ElasticSearch, añadir un GSI, o escanear y filtrar manualmente.

In [92]:
res = buscar_por_nombre_dynamo(tabla, "American")

In [93]:
res

[{'Nombre': 'American',
  'Id': Decimal('1'),
  'Stock': Decimal('11'),
  'Precio': Decimal('80.93')}]

In [ ]:
res = buscar_por_nombre_dynamo(tabla, "Exactly")
res

In [94]:
from boto3.dynamodb.conditions import Attr

def buscar_nombre_contiene_dynamo(tabla, subcadena):
    response = tabla.scan(
        FilterExpression=Attr("Nombre").contains(subcadena)
    )
    return response.get("Items", [])

In [95]:
buscar_nombre_contiene_dynamo(tabla, "A")

[{'Nombre': 'American',
  'Id': Decimal('1'),
  'Stock': Decimal('11'),
  'Precio': Decimal('80.93')},
 {'Nombre': 'Area',
  'Id': Decimal('8'),
  'Stock': Decimal('159'),
  'Precio': Decimal('63.84')},
 {'Nombre': 'Against',
  'Id': Decimal('39'),
  'Stock': Decimal('185'),
  'Precio': Decimal('56.04')},
 {'Nombre': 'Although',
  'Id': Decimal('48'),
  'Stock': Decimal('192'),
  'Precio': Decimal('49.89')},
 {'Nombre': 'Ability',
  'Id': Decimal('17'),
  'Stock': Decimal('76'),
  'Precio': Decimal('57.29')},
 {'Nombre': 'According',
  'Id': Decimal('26'),
  'Stock': Decimal('72'),
  'Precio': Decimal('35.65')},
 {'Nombre': 'Available',
  'Id': Decimal('25'),
  'Stock': Decimal('42'),
  'Precio': Decimal('44.09')},
 {'Nombre': 'Animal',
  'Id': Decimal('32'),
  'Stock': Decimal('169'),
  'Precio': Decimal('45.15')}]

🔸 ¿Qué hace contains?

Equivalente a un LIKE '%subcadena%' en SQL.

Case-sensitive. Si querés hacer una búsqueda sin distinción de mayúsculas/minúsculas, tenés que normalizar los datos (por ejemplo, guardar el nombre en minúsculas en otro campo).



🔎 Si querés que empiece con cierta cadena (startswith)


In [96]:
def buscar_nombre_empieza_con_dynamo(tabla, prefijo):
    response = tabla.scan(
        FilterExpression=Attr("Nombre").begins_with(prefijo)
    )
    return response.get("Items", [])



🧨 Consideraciones importantes

Esto no escala bien: scan lee toda la tabla, lo cual es lento y costoso en grandes volúmenes.



Para mejorar:

Podés usar un GSI (Índice Secundario Global) si el campo es muy buscado.

Para búsquedas complejas o texto libre, lo ideal es indexar con OpenSearch (antes ElasticSearch) y vincularlo con DynamoDB.

### 🔍 3. Productos con stock mayor a cierto valor

#### MongoDB

In [98]:
def productos_con_stock_mayor_mongo(db, minimo):
    return list(db.productos.find({"Stock": {"$gt": minimo}}))

In [99]:
productos_con_stock_mayor_mongo(db_mongo, 75)

[{'_id': ObjectId('6848ba0d471d6dbebeeb1b24'),
  'Id': 3,
  'Nombre': 'Not',
  'Precio': 62.9,
  'Stock': 117},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b26'),
  'Id': 5,
  'Nombre': 'Go',
  'Precio': 71.91,
  'Stock': 151},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b27'),
  'Id': 6,
  'Nombre': 'Grow',
  'Precio': 49.91,
  'Stock': 87},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b28'),
  'Id': 7,
  'Nombre': 'Together',
  'Precio': 91.85,
  'Stock': 84},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b29'),
  'Id': 8,
  'Nombre': 'Area',
  'Precio': 63.84,
  'Stock': 159},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b2b'),
  'Id': 10,
  'Nombre': 'Central',
  'Precio': 9.34,
  'Stock': 194},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b2e'),
  'Id': 13,
  'Nombre': 'Goal',
  'Precio': 40.86,
  'Stock': 168},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b30'),
  'Id': 15,
  'Nombre': 'Else',
  'Precio': 74.66,
  'Stock': 90},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b32'),
  'Id': 17,
  'Nombre': 'Ability',
  'P

#### DynamoDB


In [100]:
def productos_con_stock_mayor_dynamo(tabla, minimo):
    response = tabla.scan(
        FilterExpression=Attr("Stock").gt(minimo)
    )
    return response.get("Items", [])

In [101]:
productos_con_stock_mayor_dynamo(tabla, 75)

[{'Nombre': 'Painting',
  'Id': Decimal('43'),
  'Stock': Decimal('113'),
  'Precio': Decimal('72.39')},
 {'Nombre': 'Picture',
  'Id': Decimal('23'),
  'Stock': Decimal('117'),
  'Precio': Decimal('49.38')},
 {'Nombre': 'Blue',
  'Id': Decimal('30'),
  'Stock': Decimal('180'),
  'Precio': Decimal('35.87')},
 {'Nombre': 'Else',
  'Id': Decimal('15'),
  'Stock': Decimal('90'),
  'Precio': Decimal('74.66')},
 {'Nombre': 'Not',
  'Id': Decimal('3'),
  'Stock': Decimal('117'),
  'Precio': Decimal('62.9')},
 {'Nombre': 'Go',
  'Id': Decimal('5'),
  'Stock': Decimal('151'),
  'Precio': Decimal('71.91')},
 {'Nombre': 'Area',
  'Id': Decimal('8'),
  'Stock': Decimal('159'),
  'Precio': Decimal('63.84')},
 {'Nombre': 'Central',
  'Id': Decimal('10'),
  'Stock': Decimal('194'),
  'Precio': Decimal('9.34')},
 {'Nombre': 'Change',
  'Id': Decimal('35'),
  'Stock': Decimal('97'),
  'Precio': Decimal('56.73')},
 {'Nombre': 'Against',
  'Id': Decimal('39'),
  'Stock': Decimal('185'),
  'Precio': Deci

### 🔍 4. Productos con precio entre dos valores


#### MongoDB


In [102]:
def productos_en_rango_precio_mongo(db, minimo, maximo):
    return list(db.productos.find({
        "Precio": {"$gte": minimo, "$lte": maximo}
    }))


In [103]:
productos_en_rango_precio_mongo(db_mongo, 80, 100)

[{'_id': ObjectId('6848ba0d471d6dbebeeb1b22'),
  'Id': 1,
  'Nombre': 'American',
  'Precio': 80.93,
  'Stock': 11},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b28'),
  'Id': 7,
  'Nombre': 'Together',
  'Precio': 91.85,
  'Stock': 84},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b2d'),
  'Id': 12,
  'Nombre': 'Option',
  'Precio': 99.85,
  'Stock': 48},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b33'),
  'Id': 18,
  'Nombre': 'Western',
  'Precio': 93.43,
  'Stock': 179},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b35'),
  'Id': 20,
  'Nombre': 'Staff',
  'Precio': 85.14,
  'Stock': 100},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b37'),
  'Id': 22,
  'Nombre': 'Unit',
  'Precio': 83.37,
  'Stock': 182},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b4b'),
  'Id': 42,
  'Nombre': 'Law',
  'Precio': 92.42,
  'Stock': 71},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b4d'),
  'Id': 44,
  'Nombre': 'Me',
  'Precio': 82.85,
  'Stock': 38},
 {'_id': ObjectId('6848ba0d471d6dbebeeb1b4e'),
  'Id': 45,
  'Nombre': 'Evid

#### DynamoDB

In [104]:
def productos_en_rango_precio_dynamo(tabla, minimo, maximo):
    from decimal import Decimal
    response = tabla.scan(
        FilterExpression=Attr("Precio").between(Decimal(str(minimo)), Decimal(str(maximo)))
    )
    return response.get("Items", [])

In [105]:
productos_en_rango_precio_dynamo(tabla, 80, 100)

[{'Nombre': 'Evidence',
  'Id': Decimal('45'),
  'Stock': Decimal('18'),
  'Precio': Decimal('96.98')},
 {'Nombre': 'Option',
  'Id': Decimal('12'),
  'Stock': Decimal('48'),
  'Precio': Decimal('99.85')},
 {'Nombre': 'American',
  'Id': Decimal('1'),
  'Stock': Decimal('11'),
  'Precio': Decimal('80.93')},
 {'Nombre': 'Manager',
  'Id': Decimal('46'),
  'Stock': Decimal('45'),
  'Precio': Decimal('98.48')},
 {'Nombre': 'Together',
  'Id': Decimal('7'),
  'Stock': Decimal('84'),
  'Precio': Decimal('91.85')},
 {'Nombre': 'Unit',
  'Id': Decimal('22'),
  'Stock': Decimal('182'),
  'Precio': Decimal('83.37')},
 {'Nombre': 'Staff',
  'Id': Decimal('20'),
  'Stock': Decimal('100'),
  'Precio': Decimal('85.14')},
 {'Nombre': 'Law',
  'Id': Decimal('42'),
  'Stock': Decimal('71'),
  'Precio': Decimal('92.42')},
 {'Nombre': 'Me',
  'Id': Decimal('44'),
  'Stock': Decimal('38'),
  'Precio': Decimal('82.85')},
 {'Nombre': 'Western',
  'Id': Decimal('18'),
  'Stock': Decimal('179'),
  'Precio': 

### 🔄 ACTUALIZAR


#### MongoDB

In [ ]:
def actualizar_producto_mongo(db, id_producto, nuevos_datos):
    resultado = db.productos.update_one(
        {"Id": id_producto},
        {"$set": nuevos_datos}
    )
    return resultado.modified_count

In [ ]:
actualizar_producto_mongo(db_mongo, 1, {"Precio": 99.99, "Stock": 200})

In [ ]:
mongo_1 = buscar_producto_mongo(db_mongo, 1)
mongo_1

#### DynamoDB

In [ ]:
def actualizar_producto_dynamo(tabla, id_producto, nuevos_datos):
    
    # Construir la expresión de actualización
    expresion = "SET " + ", ".join(f"#{k}=:{k}" for k in nuevos_datos)
    print(expresion)
    # Crear los nombres y valores para la expresión
    nombres = {f"#{k}": k for k in nuevos_datos}
    print(nombres)
    # Crear los valores para la expresión
    valores = {f":{k}": v for k, v in nuevos_datos.items()}
    print(valores)
    tabla.update_item(
        Key={"Id": id_producto},
        UpdateExpression=expresion,
        ExpressionAttributeNames=nombres,
        ExpressionAttributeValues=valores
    )


In [ ]:
dynamo_1 = buscar_producto_dynamo(tabla, 1)
dynamo_1

⚠️ En DynamoDB, los números deben ser Decimal, no float.

In [ ]:
from decimal import Decimal

In [ ]:
actualizar_producto_dynamo(tabla, 1, {"Precio": Decimal("99.99"), "Stock": 200})

In [ ]:
dynamo_1 = buscar_producto_dynamo(tabla, 1)
dynamo_1

### 🗑️ BORRAR

##### 🧹 MongoDB

In [ ]:
def borrar_producto_mongo(db, id_producto):
    resultado = db.productos.delete_one({"Id": id_producto})
    return resultado.deleted_count


In [ ]:
borrar_producto_mongo(db_mongo, 1)

#### 🧹 DynamoDB

In [ ]:
def borrar_producto_dynamo(tabla, id_producto):
    tabla.delete_item(Key={"Id": id_producto})

In [ ]:
borrar_producto_dynamo(tabla, 1)

### 🔁 Iterar

#### MongoDB – Iterador natural



Mongo devuelve un cursor iterable directamente:

🔹 Lo usás así:

In [ ]:
for producto in db_mongo.productos.find():
    print(producto)



Este cursor no carga todo en memoria, así que es ideal para grandes volúmenes.

#### DynamoDB – scan() paginado manualmente


DynamoDB no devuelve un generador nativo, pero podés construir uno para iterar en páginas:

In [ ]:
def scan_dynamo_paginas(tabla):
    # Primera llamada al escaneo:
    response = tabla.scan()
    # para devolver todos los elementos obtenidos en la primera llamada al escaneo
    yield from response.get("Items", []) # esto devuelve un generador
    
    # mientras haya una clave de evaluación final (LastEvaluatedKey) en la respuesta
    while "LastEvaluatedKey" in response:
        # Escaneo de páginas adicionales:
        # pasando la clave de evaluación final como ExclusiveStartKey
        # Esto permite continuar desde donde se dejó en la llamada anterior.
        response = tabla.scan(ExclusiveStartKey=response["LastEvaluatedKey"])
        yield from response.get("Items", [])



🔹 Lo usás así:


In [ ]:
for item in scan_dynamo_paginas(tabla):
    print(item)


🔁 Esto escanea toda la tabla, de forma paginada (Dynamo devuelve máximo 1 MB por respuesta).